In [1]:
import sys, os
sys.path.append('../')
result_dir = os.path.join(os.path.dirname(os.getcwd()), 'result', 'grad_inspect')
figure_dir = os.path.join(os.path.dirname(os.getcwd()), 'figure')

from src.metric import MMD_heuristic, median_heuristic, c2st
from experiment.toggle_switch import calculate_cost
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.simulator.ts import ToggleSwitch


from torch import Tensor, nn
from torch.nn import functional as F
import torch
# from src.metric import maximum_mean_discrepancy, c2st
import matplotlib

plt.rcParams['text.latex.preamble'] = r'\usepackage{times}'

/opt/homebrew/anaconda3/envs/sbi_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
n_list =  [10000, 500, 100]
T_list =  [50, 80, 300]
C = calculate_cost(n_list, T_list)
n_str = '_'.join([str(n) for n in n_list])
T_str = '_'.join([str(T) for T in T_list])

In [3]:
net_r_p = torch.load(os.path.join(result_dir, 'ts_grad_inspect_n_10000_500_100_T_50_80_300__gradient_surgery_gradient_rescale_.pt'))
net_r = torch.load(os.path.join(result_dir, 'ts_grad_inspect_n_10000_500_100_T_50_80_300__gradient_rescale_.pt'))
neta_p = torch.load(os.path.join(result_dir, 'ts_grad_inspect_n_10000_500_100_T_50_80_300__gradient_surgery_.pt'))
net_none = torch.load(os.path.join(result_dir, 'ts_grad_inspect_n_10000_500_100_T_50_80_300__.pt'))

/var/folders/r6/jlzvq0td5mqf05q5jm6fs2vm0000gp/T/ipykernel_17123/1254293397.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net_r_p = torch.load(os.path.join(result_dir,

In [4]:
# np.save(os.path.join(result_dir, 'ts_theta.npy'), theta_test)
# np.save(os.path.join(result_dir, 'ts_x.npy'), x_test)
theta_test = np.load(os.path.join(result_dir, 'ts_theta.npy'))
x_test = np.load(os.path.join(result_dir, 'ts_x.npy'))

In [5]:
theta_test = torch.tensor(theta_test)
x_test = torch.tensor(x_test)

In [6]:
network_list = [net_r_p, net_r, neta_p, net_none]
idx = 1
length_scale = median_heuristic(x_test[idx].unsqueeze(1))

In [7]:
n = theta_test.shape[0]
m = x_test.shape[1]

In [8]:
mmd_mlmc = np.zeros((n, len(network_list)))

for i in range(n):
    x_test_i = x_test[i].unsqueeze(1)
    for j in range(len(network_list)):
        x_emu_i_j = network_list[j].sample_unstandardized(num_samples = m, condition = theta_test[i].unsqueeze(0)).squeeze(0)
        mmd_mlmc[i, j] = MMD_heuristic(x_test_i, x_emu_i_j, l_list = [length_scale])

In [13]:
np.save(os.path.join(result_dir, 'toggle_switch_grad_inspect_mmd.npy'), mmd_mlmc)
mmd_mlmc = np.load(os.path.join(result_dir, 'toggle_switch_grad_inspect_mmd.npy'))

In [14]:
name = ["(i) both", "(ii) only rescaling", "(iii) only projection", "standard"]

for i in range(len(network_list)):
    print(name[i], round(mmd_mlmc[:, i].mean(), 2), '(', round(mmd_mlmc[:, i].std(), 2), ')', '\n')


(i) both 0.26 ( 0.25 ) 

(ii) only rescaling 0.5 ( 0.37 ) 

(iii) only projection 0.35 ( 0.27 ) 

standard 0.59 ( 0.31 ) 

